In [1]:
%load_ext autoreload
%autoreload 2

### Basic Imports

In [2]:
import nbimporter
from typing import Dict, Tuple, List
import numpy as np
from tqdm import tqdm
from qiskit.opflow import X, Z, I, H, Y

### Env Vars

In [3]:
QUBITS_NUM = 4 
N = 2**QUBITS_NUM

NUM_SHOTS = 1024
NUM_ITERATIONS = 100

CIRCUIT_DEPTH = 3
PARAMS_NUM = 2*QUBITS_NUM*(CIRCUIT_DEPTH+1)
RANDOM_INITIAL_THETAS = np.random.uniform(low=0, high=360, size=PARAMS_NUM)

### Simulator Backend

In [4]:
from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals

seed = 50
algorithm_globals.random_seed = seed

simulator_backend = Aer.get_backend('qasm_simulator')

### BFGS Optimizer

In [5]:
from scipy.optimize import minimize

### Ansatz State

In [6]:
from linear_entangelment_and_full_entangelment_ansatz_circuits import *

In [7]:
def get_ansatz_state(thetas, ansatz_entangelment, input_state):
    if ansatz_entangelment=="full":
        return get_full_entangelment_ansatz(QUBITS_NUM, thetas, input_state)
    if ansatz_entangelment=="linear":
        return get_linear_entangelment_ansatz(QUBITS_NUM, thetas, input_state)

## Expectation Value

### convert hamiltonian to pauli strings

In [8]:
def transfrom_hamiltonian_into_pauli_strings(hamiltonian) -> List:
    pauli_operators = hamiltonian.to_pauli_op().settings['oplist']
    pauli_coeffs = list(map(lambda pauli_operator: pauli_operator.coeff, pauli_operators))
    pauli_strings = list(map(lambda pauli_operator: pauli_operator.primitive, pauli_operators))
    return pauli_coeffs, pauli_strings

### pauli string reduction to sigma_z's

In [9]:
from qiskit.circuit.library.standard_gates import HGate, SGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

In [10]:
reducing_to_pauli_z_mapping = {
    'I': 'I',
    'Z': 'Z',
    'X': 'Z',
    'Y': 'Z'
} 

In [11]:
def reduce_pauli_matrixes_into_sigma_z(pauli_string) -> str:
    reduced_pauli_string = ""
    for matrix_index in range(QUBITS_NUM):
        pauli_matrix = str(pauli_string[matrix_index])
        reduced_pauli_matrix = reducing_to_pauli_z_mapping[pauli_matrix]
        reduced_pauli_string = reduced_pauli_matrix + reduced_pauli_string
    
    return reduced_pauli_string

In [12]:
def add_layer_of_gates_for_reducing_paulis_to_sigma_z(pauli_string, quantum_circuit):
    quantum_registers = QuantumRegister(QUBITS_NUM, name="qubit")
    additional_circuit_layer = QuantumCircuit(quantum_registers)
    
    for quantum_register_index, pauli_matrix in enumerate(pauli_string):
        if pauli_matrix == "X":
            additional_circuit_layer.append(HGate(), [quantum_registers[quantum_register_index]])
        if pauli_string == "Y":
            additional_circuit_layer.append(HGate(), [quantum_registers[quantum_register_index]])
            additional_circuit_layer.append(SGate(), [quantum_registers[quantum_register_index]])
                
    extended_quantum_circuit = quantum_circuit.compose(additional_circuit_layer)
    return extended_quantum_circuit

### probabilities distribution

In [13]:
def get_probability_distribution(counts: Dict) -> Dict:
    proba_distribution = {state: (count / NUM_SHOTS) for state, count in counts.items()}
    return proba_distribution

def calculate_probabilities_of_measurments_in_computational_basis(quantum_state_circuit) -> Dict:
    quantum_state_circuit.measure_all()
    
    transpiled_quantum_state_circuit = transpile(quantum_state_circuit, simulator_backend) 
    Qobj = assemble(transpiled_quantum_state_circuit)
    result = simulator_backend.run(Qobj).result()
    counts = result.get_counts(quantum_state_circuit)
    
    return get_probability_distribution(counts)

### Expectation value from probabilities

In [14]:
def sort_probas_dict_by_qubits_string_keys(proba_distribution: Dict) -> Dict:
    return dict(sorted(proba_distribution.items()))

def reset_power_of_minus_1(power_of_minus_1):
    power_of_minus_1 = 0
    return power_of_minus_1

def convert_pauli_string_into_str(pauli_string) -> str:
    return str(pauli_string)

def calculate_expectation_value_of_pauli_string_by_measurments_probas(pauli_string, ansatz_circuit):
    pauli_string_expectation_value = 0
    power_of_minus_1 = 0
    
    pauli_string_str = convert_pauli_string_into_str(pauli_string)
    extended_ansatz_circuit = add_layer_of_gates_for_reducing_paulis_to_sigma_z(pauli_string_str, ansatz_circuit)
    probas_distribution = calculate_probabilities_of_measurments_in_computational_basis(extended_ansatz_circuit)
    
    reduced_pauli_string = reduce_pauli_matrixes_into_sigma_z(pauli_string)
    sorted_probas_distribuition = sort_probas_dict_by_qubits_string_keys(probas_distribution)
    for qubits_string, proba in sorted_probas_distribuition.items():
        for string_index in range(QUBITS_NUM):
            if(str(qubits_string[string_index])=="1" and str(reduced_pauli_string[string_index])=="Z"):
                power_of_minus_1 += 1
            
        pauli_string_expectation_value += pow(-1, power_of_minus_1)*proba
        power_of_minus_1 = reset_power_of_minus_1(power_of_minus_1)
        
    return pauli_string_expectation_value

In [15]:
def get_expectation_value(ansatz_circuit, pauli_coeffs, pauli_strings):
    total_expection_value = 0
    
    for pauli_coeff, pauli_string in tqdm(zip(pauli_coeffs, pauli_strings)):
        total_expection_value += pauli_coeff*calculate_expectation_value_of_pauli_string_by_measurments_probas(
                                                                                    pauli_string, ansatz_circuit)
    
    return total_expection_value

## Objective Function

In [16]:
from qiskit import assemble, transpile

def cost_function(thetas, hamiltonian, ansatz_entangelment, initial_quantum_state):
    pauli_coeffs, pauli_strings = transfrom_hamiltonian_into_pauli_strings(hamiltonian)
    ansatz_state = get_ansatz_state(thetas, ansatz_entangelment, initial_quantum_state)
        
    L = get_expectation_value(ansatz_state, pauli_coeffs, pauli_strings)
    insert_approximated_energy_to_list_of_all_approximated_energies(L)
        
    return L

## MUB

In [17]:
from typing import List
import numpy as np

class MUB:
    def __init__(self, name: str, color: str, basis_vectors):
        self.name = name
        self.color = color
        self.basis_vectors = basis_vectors
        self.approximated_eigenvalue_results = {tuple(basis_vector): 0 for basis_vector in basis_vectors}
        self.approximation_error_results = {tuple(basis_vector): 0 for basis_vector in basis_vectors}
        self.iterations_number = {tuple(basis_vector): 0 for basis_vector in basis_vectors}
        self.lists_of_approximated_energies_till_convergence = {tuple(basis_vector): [] for basis_vector in basis_vectors}
        

In [18]:
M0_vectors = np.identity(4)

M1_vectors = [np.vectorize(complex)([0.5,0.5,0.5,0.5]),
      np.vectorize(complex)([0.5,0.5,-0.5,-0.5]),
      np.vectorize(complex)([0.5,-0.5,-0.5,0.5]),
      np.vectorize(complex)([0.5,-0.5,0.5,-0.5])]

M2_vectors = [np.vectorize(complex)([0.5,0.5,0,0],[0,0,-0.5,-0.5]),
      np.vectorize(complex)([0.5,-0.5,0,0],[0,0,0.5,0.5]),
      np.vectorize(complex)([0.5,0.5,0,0],[0,0,0.5,-0.5]),
      np.vectorize(complex)([0.5,0.5,0,0],[0,0,-0.5,0.5])]

M3_vectors = [np.vectorize(complex)([0.5,0,0,-0.5],[0,-0.5,-0.5,-0]),
      np.vectorize(complex)([0.5,0,0,0.5],[0,-0.5,0.5,0]),
      np.vectorize(complex)([0.5,0,0,-0.5],[0,0.5,0.5,0]),
      np.vectorize(complex)([0.5,0,0,0.5],[0,0.5,-0.5,0])]

M4_vectors = [np.vectorize(complex)([0.5,0,-0.5,0],[0,-0.5,0,-0.5]),
      np.vectorize(complex)([0.5,0,0.5,0],[0,-0.5,0,0.5]),
      np.vectorize(complex)([0.5,0,-0.5,0],[0,0.5,0,0.5]),
      np.vectorize(complex)([0.5,0,0.5,0],[0,0.5,0,-0.5])]

In [19]:
v_00 = np.vectorize(complex)([1,0,0,0])

def extend_vector_with_00_tensor_product(vector):
    return np.kron(v_00, vector)

In [20]:
M0 = MUB(name='M0', color='b', basis_vectors=[extend_vector_with_00_tensor_product(vector) for vector in M0_vectors])
M1 = MUB(name='M1', color='c', basis_vectors=[extend_vector_with_00_tensor_product(vector) for vector in M1_vectors])
M2 = MUB(name='M2', color='y', basis_vectors=[extend_vector_with_00_tensor_product(vector) for vector in M2_vectors])
M3 = MUB(name='M3', color='m', basis_vectors=[extend_vector_with_00_tensor_product(vector) for vector in M3_vectors])
M4 = MUB(name='M4', color='r', basis_vectors=[extend_vector_with_00_tensor_product(vector) for vector in M4_vectors])

mubs_d_4 = [M0, M1, M2, M3, M4]

## Optimization

In [21]:
def get_approximated_eigenvalue_of_hamiltonian(hamiltonian, ansatz_entangelment, initial_quantum_state):
    optimizer_result = minimize(cost_function,
                                x0=RANDOM_INITIAL_THETAS,
                                args=(hamiltonian, ansatz_entangelment, initial_quantum_state),
                                method="COBYLA")
    optimal_thetas = optimizer_result.x
    approximated_eigenvalue = optimizer_result.fun
     
    return approximated_eigenvalue

## Comparsion

In [22]:
from numpy import linalg as LA

def get_approximation_error(exact_eigenvalue, approximated_eigenvalue):
    return abs(abs(exact_eigenvalue)-abs(approximated_eigenvalue))/abs(exact_eigenvalue)

In [23]:
def get_minimum_exact_eigenvalue_of_hamiltonian(hamiltonian):
    eigen_values = LA.eigvals(hamiltonian.to_matrix())
    
    return min(sorted(eigen_values))

In [24]:
def compare_exact_and_approximated_eigenvalue(hamiltonian, approximated_eigenvalue):
    exact_eigenvalue = get_minimum_exact_eigenvalue_of_hamiltonian(hamiltonian)
    print("Exact Eigenvalue:")
    print(exact_eigenvalue)
    print("\nApproximated Eigenvalue:")
    print(approximated_eigenvalue)

    print("\nApproximation Error")
    print(get_approximation_error(exact_eigenvalue, approximated_eigenvalue))
    
    plot_convergence_of_optimization_process(approximated_energies, exact_eigenvalue, margin=3)

## Visualization

In [25]:
global approximated_energies
approximated_energies = []

In [26]:
def insert_approximated_energy_to_list_of_all_approximated_energies(energy):
    approximated_energies.append(energy)

In [27]:
import matplotlib.pyplot as plt

def plot_convergence_of_optimization_process(approximated_energies, exact_eigenvalue, margin):
    plt.title("convergence of optimization process to the exact eigenvalue")
    plt.margins(0, margin)
#     plt.plot(approximated_energies[-NUM_ITERATIONS:])
    plt.plot(approximated_energies)
    plt.axhline(y = exact_eigenvalue, color = 'r', linestyle = '-')
    plt.grid()
    plt.xlabel("# of iterations")
    plt.ylabel("Energy")

## H2 Molecule 4 qubits

In [28]:
H2_molecule_Hamiltonian_4_qubits =  -0.8105479805373279 * (I^I^I^I) \
                                    + 0.1721839326191554 * (I^I^I^Z) \
                                    - 0.22575349222402372 * (I^I^Z^I) \
                                    + 0.17218393261915543 * (I^Z^I^I) \
                                    - 0.2257534922240237 * (Z^I^I^I) \
                                    + 0.12091263261776627 * (I^I^Z^Z) \
                                    + 0.16892753870087907 * (I^Z^I^Z) \
                                    + 0.045232799946057826 * (Y^Y^Y^Y) \
                                    + 0.045232799946057826 * (X^X^Y^Y) \
                                    + 0.045232799946057826 * (Y^Y^X^X) \
                                    + 0.045232799946057826 * (X^X^X^X) \
                                    + 0.1661454325638241 * (Z^I^I^Z) \
                                    + 0.1661454325638241 * (I^Z^Z^I) \
                                    + 0.17464343068300453 * (Z^I^Z^I) \
                                    + 0.12091263261776627 * (Z^Z^I^I)

#### Linear Entangelment

In [29]:
# %%time
for basis in mubs_d_4:
    for quantum_state in basis.basis_vectors:
        H2_approximated_eigenvalue = get_approximated_eigenvalue_of_hamiltonian(H2_molecule_Hamiltonian_4_qubits, "linear", quantum_state)
        basis.approximated_eigenvalue_results[tuple(quantum_state)] = H2_approximated_eigenvalue
        iterations_num = len(approximated_energies)
        print("number of iter is {0}".format(iterations_num))
        basis.iterations_number[tuple(quantum_state)] = iterations_num
        basis.lists_of_approximated_energies_till_convergence[(tuple(quantum_state))].append(approximated_energies)
        
        exact_eigenvalue = get_minimum_exact_eigenvalue_of_hamiltonian(H2_molecule_Hamiltonian_4_qubits)
        basis.approximation_error_results[(tuple(quantum_state))] = get_approximation_error(exact_eigenvalue, H2_approximated_eigenvalue)
        
        x_values = range(1, iterations_num + 1)
        plt.plot(x_values, approximated_energies, label='Quantum Vector {0}'.format(str(quantum_state)), marker='o')  # Added markers for clarity
        approximated_energies = []

# Adding labels and legend
plt.xlabel('Number of Iterations till Convergence')
plt.ylabel('Approximated Energies')
plt.title('Approximated Energies vs. Number of Iterations')
plt.legend()

# Display the plot
plt.show()

15it [00:03,  4.55it/s]
15it [00:01,  9.58it/s]
15it [00:01,  9.14it/s]
15it [00:01, 10.09it/s]
15it [00:01, 10.38it/s]
15it [00:01, 10.25it/s]
15it [00:01, 10.44it/s]
15it [00:01, 10.13it/s]
15it [00:01, 10.48it/s]
15it [00:01, 10.20it/s]
15it [00:01, 10.65it/s]
15it [00:01, 10.05it/s]
15it [00:01,  9.45it/s]
15it [00:01, 10.02it/s]
15it [00:01, 10.14it/s]
15it [00:01, 10.30it/s]
15it [00:01,  9.50it/s]
15it [00:01, 10.18it/s]
15it [00:01,  9.78it/s]
15it [00:01, 10.02it/s]
15it [00:01, 10.20it/s]
15it [00:01,  9.97it/s]
15it [00:01,  9.98it/s]
15it [00:01,  9.93it/s]
15it [00:01,  9.96it/s]
15it [00:01, 10.14it/s]
15it [00:01,  9.70it/s]
15it [00:01,  9.68it/s]
15it [00:01,  9.70it/s]
15it [00:01, 10.52it/s]
15it [00:01, 10.00it/s]
15it [00:01, 10.02it/s]
15it [00:01, 10.27it/s]
15it [00:01, 10.20it/s]
15it [00:01,  8.98it/s]
15it [00:01,  9.50it/s]
15it [00:01,  9.52it/s]
15it [00:01,  9.92it/s]
15it [00:01,  7.73it/s]
15it [00:01,  8.61it/s]
15it [00:01,  8.95it/s]
15it [00:01,  8.

number of iter is 288


NameError: name 'qunatum_state' is not defined

In [ ]:
def generate_approximation_errors(hamiltonian):
    for basis in mubs_d_4:
        for vector, approx_val in basis.approximated_eigenvalue_results.items():
            exact_eigenvalue = get_minimum_exact_eigenvalue_of_hamiltonian(hamiltonian)
            basis.approximation_error_results[vector] = get_approximation_error(exact_eigenvalue, approx_val)

In [ ]:
generate_approximation_errors(H2_molecule_Hamiltonian_4_qubits)

In [ ]:
import statistics

values = []
for basis in mubs_d_4:
    for value in basis.approximated_eigenvalue_results.values():
        values.append(value)

# Calculating the mean and median
mean_value = statistics.mean(values)
median_value = statistics.median(values)

print(f"Mean: {mean_value}")
print(f"Median: {median_value}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Number of series
num_series = 5

# Plotting each series of energies generated within the loop
for i in range(num_series):
    # Generate a series of approximated energies with varying lengths
    # For demonstration, let's create a list of energies with increasing length and varying values
    energy_series = np.random.rand(i + 3) * 10  # Random energies, length varies in each iteration
    
    # The x-values are the number of iterations till convergence, which is just the index of each energy value
    x_values = range(1, len(energy_series) + 1)
    
    # Plotting the series on the same graph
    plt.plot(x_values, energy_series, label=f'Series {i + 1}', marker='o')  # Added markers for clarity

# Adding labels and legend
plt.xlabel('Number of Iterations till Convergence')
plt.ylabel('Approximated Energies')
plt.title('Approximated Energies vs. Number of Iterations')
plt.legend()

# Display the plot
plt.show()


In [ ]:
compare_exact_and_approximated_eigenvalue(H2_molecule_Hamiltonian_4_qubits, H2_approximated_eigenvalue)

#### Full Entangelment

In [ ]:
%%time
H2_approximated_eigenvalue = get_approximated_eigenvalue_of_hamiltonian(H2_molecule_Hamiltonian_4_qubits, "full")

In [ ]:
compare_exact_and_approximated_eigenvalue(H2_molecule_Hamiltonian_4_qubits, H2_approximated_eigenvalue)

##  Transverse Ising Model 4 qubits

In [ ]:
transverse_ising_4_qubits = 0.0 * (I^I^I^I) \
    + 0.8398088405253477 * (X^I^I^I) \
    + 0.7989496312070936 * (I^X^I^I) \
    + 0.38189710487113193 * (Z^Z^I^I) \
    + 0.057753122422666725 * (I^I^X^I) \
    + 0.5633292636970458 * (Z^I^Z^I) \
    + 0.3152740621483513 * (I^Z^Z^I) \
    + 0.07209487981989715 * (I^I^I^X) \
    + 0.17892334004292654 * (Z^I^I^Z) \
    + 0.2273896497668042 * (I^Z^I^Z) \
    + 0.09762902934216211 * (I^I^Z^Z)

#### Linear Entangelment

In [ ]:
%%time
TI_approximated_eigenvalue = get_approximated_eigenvalue_of_hamiltonian(transverse_ising_4_qubits, "linear")

In [ ]:
compare_exact_and_approximated_eigenvalue(transverse_ising_4_qubits, TI_approximated_eigenvalue)

#### Full Entangelment

In [ ]:
%%time
TI_approximated_eigenvalue = get_approximated_eigenvalue_of_hamiltonian(transverse_ising_4_qubits, "full")

In [ ]:
compare_exact_and_approximated_eigenvalue(transverse_ising_4_qubits, TI_approximated_eigenvalue)

##  Transverse Ising Model 3 qubits

In [ ]:
QUBITS_NUM = 3 
N = 2**QUBITS_NUM

NUM_SHOTS = 1024
NUM_ITERATIONS = 100

CIRCUIT_DEPTH = 3
PARAMS_NUM = 2*QUBITS_NUM*(CIRCUIT_DEPTH+1)

In [ ]:
from qiskit.opflow import X, Z, I

transverse_ising_3_qubits = 0.0 * (I^I^I) \
                    + 0.012764169333459807 * (X^I^I) \
                    + 0.7691573729160869 * (I^X^I) \
                    + 0.398094746026449 * (Z^Z^I) \
                    + 0.15250261906586637 * (I^I^X) \
                    + 0.2094051920882264 * (Z^I^Z) \
                    + 0.5131291860752999 * (I^Z^Z)

#### Linear Entangelment

In [ ]:
%%time
TI_approximated_eigenvalue = get_approximated_eigenvalue_of_hamiltonian(transverse_ising_3_qubits, "linear")

In [ ]:
compare_exact_and_approximated_eigenvalue(transverse_ising_3_qubits, TI_approximated_eigenvalue)

#### Full Entangelment

In [ ]:
%%time
TI_approximated_eigenvalue = get_approximated_eigenvalue_of_hamiltonian(transverse_ising_3_qubits, "full")

In [ ]:
compare_exact_and_approximated_eigenvalue(transverse_ising_3_qubits, TI_approximated_eigenvalue)

##  Transverse Ising Model 2 qubits

In [ ]:
QUBITS_NUM = 2 
N = 2**QUBITS_NUM

NUM_SHOTS = 1024
NUM_ITERATIONS = 100

CIRCUIT_DEPTH = 3
PARAMS_NUM = 2*QUBITS_NUM*(CIRCUIT_DEPTH+1)

In [ ]:
transverse_ising_2_qubits = 0.13755727363376802 * (I^X) \
                            + 0.43305656297810435 * (X^I) \
                            + 0.8538597608997253 * (Z^Z)

#### Linear Entangelment

In [ ]:
%%time
TI_approximated_eigenvalue = get_approximated_eigenvalue_of_hamiltonian(transverse_ising_2_qubits, "linear")

In [ ]:
compare_exact_and_approximated_eigenvalue(transverse_ising_2_qubits, TI_approximated_eigenvalue)

#### Full Entangelment

In [ ]:
%%time
TI_approximated_eigenvalue = get_approximated_eigenvalue_of_hamiltonian(transverse_ising_2_qubits, "full")

In [ ]:
compare_exact_and_approximated_eigenvalue(transverse_ising_2_qubits, TI_approximated_eigenvalue)

## H2 Molecule 2 qubits

In [ ]:
from qiskit.opflow import X, Z, I

H2_molecule_Hamiltonian_2_qubits = -0.5053051899926562*(I^I) + \
                            -0.3277380754984016*(Z^I) + \
                            0.15567463610622564*(Z^Z) + \
                            -0.3277380754984016*(I^Z)

#### Linear Entangelment

In [ ]:
%%time
H2_approximated_eigenvalue = get_approximated_eigenvalue_of_hamiltonian(H2_molecule_Hamiltonian_2_qubits, "linear")

In [ ]:
compare_exact_and_approximated_eigenvalue(H2_molecule_Hamiltonian_2_qubits, H2_approximated_eigenvalue)

#### Full Entangelment

In [ ]:
%%time
H2_approximated_eigenvalue = get_approximated_eigenvalue_of_hamiltonian(H2_molecule_Hamiltonian_2_qubits, "full")

In [ ]:
compare_exact_and_approximated_eigenvalue(H2_molecule_Hamiltonian_2_qubits, H2_approximated_eigenvalue)